In [1]:
from collections import defaultdict
from datetime import datetime

import pandas as pd
import numpy as np
'''
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
'''

'\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.metrics import classification_report\n'

In [2]:
log_data = pd.read_csv('Logs.csv',sep = ';')

In [3]:
print(log_data.head())

              log_date       log_icon             log_time       log_type  \
0  Tuesday, 2024-04-30  Service alert  2024-04-30 23:59:07  SERVICE ALERT   
1  Tuesday, 2024-04-30  Service alert  2024-04-30 23:54:21  SERVICE ALERT   
2  Tuesday, 2024-04-30  Service alert  2024-04-30 23:49:34  SERVICE ALERT   
3  Tuesday, 2024-04-30  Service alert  2024-04-30 23:49:32  SERVICE ALERT   
4  Tuesday, 2024-04-30  Service alert  2024-04-30 23:49:07  SERVICE ALERT   

                        host       service_description  log_state_info  \
0  trgitamiflk42.targa.local                    Memory  HARD (WARNING)   
1  trgitamiapp12.targa.local         Number of threads       HARD (OK)   
2  pgtitamigtw02.targa.local  Check_MK HW/SW Inventory       HARD (OK)   
3  pgtitamigtw02.targa.local                  Check_MK       HARD (OK)   
4  trgitamiflk42.targa.local                    Memory  SOFT (WARNING)   

                                   log_plugin_output  
0  Total virtual memory: 79.46% - 21.

In [4]:
log_data['log_time'] = pd.to_datetime(log_data['log_time'])

In [5]:
max_uptime_hosts = log_data.groupby('host')
print(max_uptime_hosts)

In [6]:
sorted_max_uptime_hosts=max_uptime_hosts.apply(lambda x: x.sort_values(by='log_time'))

In [7]:
hosts = log_data['host'].unique()

In [8]:
for host in hosts:
    #host = 'trgitamirds02.targa.local'
    #print(host)
    #print(sorted_max_uptime_hosts.loc[host])
    #print(sorted_max_uptime_hosts.loc[host][sorted_max_uptime_hosts.loc[host]['log_state_info']=='HARD (OK)'])
    #print('****************************************************')
    system_down_state = ['HARD (CRITICAL)', 'HARD (UNKNOWN)']
    #print(sorted_max_uptime_hosts.loc[host][sorted_max_uptime_hosts.loc[host]['log_state_info']!='HARD (OK)'])
    #print(sorted_max_uptime_hosts.loc[host][sorted_max_uptime_hosts.loc[host]['log_state_info'].isin(system_down_state)])
    status_ok_df = sorted_max_uptime_hosts.loc[host][sorted_max_uptime_hosts.loc[host]['log_state_info']=='HARD (OK)']
    filtered_data = sorted_max_uptime_hosts.loc[host][sorted_max_uptime_hosts.loc[host]['log_state_info'].isin(system_down_state)]
    
    
    if not filtered_data.empty:
        #print(host)
        #print(status_ok_df)
        #print('****************************************************')
        #print(filtered_data)
        #print(status_ok_df['log_time'].diff().describe())
        
        # Filter status_ok_df
        filtered_rows_status_ok = []
        prev_row_status_ok = None
        for index, row in status_ok_df.iterrows():
            if prev_row_status_ok is not None:
                time_diff = row['log_time'] - prev_row_status_ok['log_time']
                if time_diff >= pd.Timedelta(hours=1):
                    filtered_rows_status_ok.append(row)
            else:
                filtered_rows_status_ok.append(row)
            prev_row_status_ok = row
        filtered_status_ok_df = pd.DataFrame(filtered_rows_status_ok)
        
        # Filter filtered_data
        filtered_rows_filtered_data = []
        prev_row_filtered_data = None
        for index, row in filtered_data.iterrows():
            if prev_row_filtered_data is not None:
                time_diff = row['log_time'] - prev_row_filtered_data['log_time']
                if time_diff >= pd.Timedelta(hours=1):
                    filtered_rows_filtered_data.append(row)
            else:
                filtered_rows_filtered_data.append(row)
            prev_row_filtered_data = row
        filtered_filtered_data = pd.DataFrame(filtered_rows_filtered_data)
        
        #print("Filtered status_ok_df:")
        #print(filtered_status_ok_df)
        #print("Filtered filtered_data:")
        #print(filtered_filtered_data)
        
        
        # Assuming 'log_time' column is in datetime format in both dataframes
        time_differences = []
        
        # Iterate through each row in both dataframes simultaneously
        for index, row_status_ok in filtered_status_ok_df.iterrows():
            #print(index)
            #print('*********')
            #print(row_status_ok)
            # Find the nearest corresponding row in filtered_filtered_data based on log_time
            nearest_row_index = (filtered_filtered_data['log_time'] - row_status_ok['log_time']).abs().idxmin()
            #print('nearest_row_index',nearest_row_index)
            row_filtered_data = filtered_filtered_data.loc[nearest_row_index]
            #print('row_filtered_data',row_filtered_data)
            # Calculate the time difference between log_time in both rows
            diff = row_filtered_data['log_time'] - row_status_ok['log_time']
            #print('*//**//**//**//**')
            #print('diff')
            #print(diff)
            time_differences.append(diff)
            
            #print('time_differences')
            #print(time_differences)
            
            
        # Convert the list of time differences to a numpy array for easier manipulation
        time_differences_array = np.array(time_differences)
        
        if len(time_differences_array) == 0:
            continue
        
        #print("Time differences between corresponding rows:")
        #print(time_differences_array)
        
        '''
        # Filter out negative values
        non_negative_time_differences_array = time_differences_array[time_differences_array >= np.timedelta64(0, 'ns')]
        
        print("Time differences between corresponding rows (excluding negative values):")
        print(non_negative_time_differences_array)
        '''
        # Create a boolean mask to filter out negative values
        mask = time_differences_array >= pd.Timedelta(0)
        #mask = time_differences_array >= np.timedelta64(0)
        
        # Filter out negative values using the mask
        non_negative_time_differences_array = time_differences_array[mask]
        
        #print("Time differences between corresponding rows (excluding negative values):")
        #print(non_negative_time_differences_array)
        
        if len(non_negative_time_differences_array) >0:
            print('Maximum Server Uptime for ', host)
            print(non_negative_time_differences_array.max())



        #break
    
    #break

Maximum Server Uptime for  pgtitamigtw02.targa.local
5 days 15:50:59
Maximum Server Uptime for  pgtitamigtw01.targa.local
7 days 18:00:01
Maximum Server Uptime for  trgitamiflt02.targa.local
4 days 14:59:57
Maximum Server Uptime for  trgitamit2q02.targa.local
5 days 12:05:52
Maximum Server Uptime for  trgitamiafl01.targa.local
0 days 14:05:15
Maximum Server Uptime for  trgitamikfk41.targa.local
1 days 23:56:42
Maximum Server Uptime for  trgitamikfk97.targa.local
13 days 03:16:26
Maximum Server Uptime for  trgitamimdf01
2 days 22:57:44
Maximum Server Uptime for  trgitamikfk98.targa.local
14 days 02:59:38
Maximum Server Uptime for  trgitamikfk99.targa.local
9 days 19:53:35
Maximum Server Uptime for  trgitamigtw01.targa.local
5 days 00:16:33
Maximum Server Uptime for  trgitamigtw02.targa.local
16 days 04:44:57
Maximum Server Uptime for  3PAR-UBIEST-8200
0 days 00:00:57
Maximum Server Uptime for  trgitamibal14.targa.local
19 days 14:43:41
Maximum Server Uptime for  trgitamiflt01.targa.loca